In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.optim as optim
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import collections
from collections import OrderedDict

## User Inputs

In [ ]:
infile = 'vector_stand_dataset.npz'
batch_size = 250
n_epochs = 500
valid_size = 2000
model_path = 'v_stand_model'
fig_path = 'v_stand_fig.png'
fig2_path = 'v_stand_fig_zoom.png'
df_path = 'v_stand_df'
seed = np.random.randint(low=0, high=1000000, size=1)
std_scheme = 0    #choose 0 for none, 1 for element-wise, 2 for vector-wise

## Standardization Schemes

In [ ]:
def no_std(infile):
    
    # load data
    npfile = np.load(infile)
    inputs = npfile['inputs']
    outputs = npfile['outputs']
    
    # convert to tensors
    inputs = torch.from_numpy(inputs).float()
    outputs = torch.from_numpy(outputs).float()
    
    # return dataset
    trainset = torch.utils.data.TensorDataset(inputs, outputs)
    
    return trainset


def std_elementwise(infile):
    
    # load data
    npfile = np.load(infile)
    inputs = npfile['inputs']
    outputs = npfile['outputs']

    # standardize inputs, outputs by element and covert to tensors
    inputMeans = inputs[0:int(inputs.shape[0]),:].mean(axis=0)
    inputStdDevs = inputs[0:int(inputs.shape[0]),:].std(axis=0)
    inputs = (inputs-inputMeans)/inputStdDevs
    inputs = torch.from_numpy(inputs).float()

    outputMeans = outputs[0:int(outputs.shape[0]),:].mean(axis=0)
    outputStdDevs = outputs[0:int(outputs.shape[0]),:].std(axis=0)
    outputs = (outputs-outputMeans)/outputStdDevs
    outputs = torch.from_numpy(outputs).float()
    
    # return dataset
    trainset = torch.utils.data.TensorDataset(inputs, outputs)
    
    return trainset
    

def std_vectorwise(infile):
    
    # load data
    npfile = np.load(infile)
    inputs = npfile['inputs']
    outputs = npfile['outputs']
    
    # standardize inputs, outputs by vector and covert to tensors
    inputMean = np.mean(inputs)
    inputStdDev = np.mean(inputs)
    inputs = (inputs-inputMean)/inputStdDev
    inputs = torch.from_numpy(inputs).float()

    outputMean = np.mean(outputs)
    outputStdDev = np.mean(outputs)
    outputs = (outputs-outputMean)/outputStdDev
    outputs = torch.from_numpy(outputs).float()
    
    # return dataset
    trainset = torch.utils.data.TensorDataset(inputs, outputs)
    
    return trainset

In [ ]:
# perform the selected standardization scheme
if std_scheme == 0:
    trainset = no_std(infile)  
elif std_scheme == 1:
    trainset = std_elementwise(infile)
elif std_scheme == 2:
    trainset = std_vectorwise(infile)   

## Network Definition

In [ ]:
network = OrderedDict([('lin1', nn.Linear(6, 3)),('relu1', nn.ReLU()),('lin2', nn.Linear(3, 3))]) 

model = nn.Sequential(network)

## Function Definitions

In [ ]:
# defines the training process for one epoch, returns training loss for given epoch
def train(model, loader, optimizer, criterion):
    running_train_loss = 0.0

    # put model into train mode
    model.train()

    for batch_idx, (inputs, outputs) in enumerate(loader):
        inputs_var = inputs
        outputs_var = outputs
        
        # get model output & loss for each given input
        model_outputs = model(inputs_var)
        loss = criterion(model_outputs, outputs_var)

        # record cummulative loss
        running_train_loss += loss.item()

        # gradient, optimizer steps
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    return running_train_loss


# defines the validation process for one epoch, returns validation loss for given epoch
def validate(model, loader, criterion):
    running_valid_loss = 0.0

    # put model in evaluation mode
    model.eval()

    for batch_idx, (inputs, outputs) in enumerate(loader):
        with torch.no_grad():
            inputs_var = inputs
            outputs_var = outputs

            # get model output & loss for each given input
            model_outputs = model(inputs_var)
            loss = criterion(model_outputs, outputs_var)

        # record cummulative loss
        running_valid_loss += loss.item()

    return running_valid_loss


# runs training and validation process over all epochs, returns results
def run_training(model, modelpath, figpath, fig2path, dfpath, trainset, validsize, numepochs, batchsize, seed):
    # set seed
    torch.manual_seed(seed)

    # create validation split
    indices = torch.randperm(len(trainset))
    train_indices = indices[:len(indices) - valid_size]
    train_sampler = torch.utils.data.sampler.SubsetRandomSampler(train_indices)
    valid_indices = indices[len(indices) - valid_size:]
    valid_sampler = torch.utils.data.sampler.SubsetRandomSampler(valid_indices)

    # define data loaders
    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, sampler=train_sampler)
    valid_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, sampler=valid_sampler)
  
    # set criterion, optimizer
    criterion = torch.nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters())
    
    # store results
    best_model = copy.deepcopy(model.state_dict())
    train_loss_results = []
    valid_loss_results = []
    epochs = []
    
    # train model
    for epoch in enumerate(range(n_epochs)):
        trainloss = train(model=model,loader=train_loader,criterion=criterion,optimizer=optimizer)
        print('train loss for epoch {index} attained: {loss}'.format(index=epoch[0], loss=trainloss))
        
        validloss = validate(model=model,loader=valid_loader,criterion=criterion)
        print('valid loss for epoch {index} attained: {loss}'.format(index=epoch[0], loss=validloss))
        
        train_loss_results.append(trainloss)
        valid_loss_results.append(validloss)
        epochs.append(epoch[0]+1)
        
        # check if model is the best, save if best
        if epoch[0] == 0:
            bestloss = validloss

        if epoch[0] > 0:
            if validloss < bestloss:
                bestloss = validloss
                best_model = copy.deepcopy(model)
                best_epoch = epoch[0]
                print('new best model saved')
                
        print('epoch {index} done'.format(index=epoch[0]))
        
    print('finished looping epochs')
    print('best valid loss = {}, epoch {}'.format(bestloss, best_epoch))

    # load and save the best model
    torch.save(best_model, model_path)
    print('best model loaded and saved')

    # plot training & validation loss vs. epoch
    fig1 = plt.figure()
    plt.plot(epochs, train_loss_results)
    plt.plot(epochs, valid_loss_results)
    plt.legend(['Training Loss', 'Validation Loss'], loc='upper left')
    plt.title('Model Training and Validation Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.show()
    fig1.savefig(fig_path)
    print('plot saved')
    
    # plot training & validation loss vs. epoch -- scale 2
    fig2 = plt.figure()
    plt.plot(epochs, train_loss_results)
    plt.plot(epochs, valid_loss_results)
    plt.legend(['Training Loss', 'Validation Loss'], loc='upper left')
    plt.title('Model Training and Validation Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.ylim(0,(bestloss*100))
    plt.show()
    fig2.savefig('fig2_path')
    print('plot saved')
    
    # create dataframe of epochs, losses
    d = {'trainloss':train_loss_results, 'validloss':valid_loss_results}
    df = pd.DataFrame(d, index=epochs)
    df.to_csv(df_path)
    print('dataframe saved')
    
    return

## Run Training Function

In [ ]:
# train the model
run_training(model=model, modelpath=model_path, figpath=fig_path, fig2path = fig2_path, dfpath=df_path, trainset=trainset, 
      validsize=valid_size, numepochs=n_epochs, batchsize=batch_size, seed=seed)

print('Finished!')

In [ ]:
# getting model parameters
list(model.parameters())

## Test Vector

In [ ]:
# testing the model with a chosen vector
vector1 = np.array([0.5,0.5,0.5])
vector2 = np.array([0.5,0.5,0.5])

test_input = np.append(vector1,vector2)

# standardize the input vector
if std_scheme == 0:
    test_input = torch.tensor(test_input).float()

elif std_scheme == 1:
    npfile = np.load(infile)
    inputs = npfile['inputs']
    
    inputMeans = inputs[0:int(inputs.shape[0]),:].mean(axis=0)
    inputStdDevs = inputs[0:int(inputs.shape[0]),:].std(axis=0)
    test_input = (test_input-inputMeans)/inputStdDevs
    test_input = torch.tensor(test_input).float()
    
elif std_scheme == 2:
    npfile = np.load(infile)
    inputs = npfile['inputs']
    
    inputMean = np.mean(inputs)
    inputStdDev = np.mean(inputs)
    test_input = (test_input-inputMean)/inputStdDev
    test_input = torch.tensor(test_input).float()

# put standardized input vector through model
model.eval()
test_output = model(test_input)
test_output = test_output.detach().numpy()

# unstandardizing the output
if std_scheme == 0:
    test_result = test_output  
    
elif std_scheme == 1:
    npfile = np.load(infile)
    outputs = npfile['outputs']
    
    outputMeans = outputs[0:int(outputs.shape[0]),:].mean(axis=0)
    outputStdDevs = outputs[0:int(outputs.shape[0]),:].std(axis=0)
    
    test_result = (test_output*outputStdDevs)+outputMeans
    
elif std_scheme == 2:
    npfile = np.load(infile)
    outputs = npfile['outputs']
    
    outputMean = np.mean(outputs)
    outputStdDev = np.mean(outputs)
    
    test_result = (test_output*outputStdDev)+outputMean
    
print(test_result)